In [1]:
!apt-get update -qq
!apt-get install -y aria2


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.3 [45.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1,086 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1,513 kB in 3s (546 kB/s)
Selecting previously unselected package libc-ares2:amd64.

In [2]:
!mkdir -p /content/dataset

file_urls = """
https://zenodo.org/records/13951614/files/20241016_132819_multilabel_annotations.csv
https://zenodo.org/records/13951614/files/20241018_segmentation_annotations.json
https://zenodo.org/records/13951614/files/gbif_class_mapping.csv
https://zenodo.org/records/13951614/files/images.zip
"""

with open("/content/file_list.txt", "w") as f:
    f.write(file_urls.strip())

import subprocess

cmd = [
    "aria2c",
    "--max-connection-per-server=16",
    "--split=32",
    "--min-split-size=1M",
    "--max-concurrent-downloads=16",
    "--continue=true",
    "--dir=/content/dataset",
    "--input-file=/content/file_list.txt",
]

process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in process.stdout:
    print(line, end='')

!rm -rf /content/file_list.txt


08/04 13:05:33 [NOTICE] Downloading 4 item(s)

08/04 13:05:34 [NOTICE] Download complete: /content/dataset/gbif_class_mapping.csv
[DL:104KiB][#8c0c67 0B/2.1MiB(0%)][#6971af 0B/334MiB(0%)][#df3aef 0B/33GiB(0%)]
[DL:814KiB][#8c0c67 320KiB/2.1MiB(14%)][#6971af 256KiB/334MiB(0%)][#df3aef 304KiB/33GiB(0%)]
[DL:2.9MiB][#8c0c67 848KiB/2.1MiB(39%)][#6971af 2.5MiB/334MiB(0%)][#df3aef 2.7MiB/33GiB(0%)]
[DL:5.3MiB][#8c0c67 1.4MiB/2.1MiB(71%)][#6971af 7.1MiB/334MiB(2%)][#df3aef 7.8MiB/33GiB(0%)]
[DL:6.7MiB][#8c0c67 1.8MiB/2.1MiB(86%)][#6971af 12MiB/334MiB(3%)][#df3aef 13MiB/33GiB(0%)]

08/04 13:05:39 [NOTICE] Download complete: /content/dataset/20241016_132819_multilabel_annotations.csv
[DL:7.6MiB][#6971af 18MiB/334MiB(5%)][#df3aef 18MiB/33GiB(0%)]
[DL:10MiB][#6971af 35MiB/334MiB(10%)][#df3aef 29MiB/33GiB(0%)]
[DL:18MiB][#6971af 82MiB/334MiB(24%)][#df3aef 44MiB/33GiB(0%)]
[DL:19MiB][#6971af 101MiB/334MiB(30%)][#df3aef 56MiB/33GiB(0%)]
[DL:20MiB][#6971af 119MiB/334MiB(35%)][#df3aef 67MiB/33GiB(0%)

In [3]:
!rm -rf /content/sample_data
!rm /content/dataset/20241016_132819_multilabel_annotations.csv
!rm /content/dataset/20241018_segmentation_annotations.json
!rm /content/dataset/gbif_class_mapping.csv

In [4]:
!unzip /content/dataset/images.zip -d /content/dataset/images

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/images/2024_dataset/20150619_REU-St-Gilles_scuba_G0014829.JPG  
  inflating: /content/dataset/images/2024_dataset/20150925_REU-La-Saline_scuba_G0026905.JPG  
  inflating: /content/dataset/images/2024_dataset/20180318_MUS-Pointe-Esny_kite_G0031079.JPG  
  inflating: /content/dataset/images/2024_dataset/20230704_REU-Cap-La-Houssaye_scuba_5_188.jpeg  
  inflating: /content/dataset/images/2024_dataset/20150925_REU-La-Saline_scuba_G0016567.JPG  
  inflating: /content/dataset/images/2024_dataset/20151210_REU-St-Gilles_scuba_GOPR0331.JPG  
  inflating: /content/dataset/images/2024_dataset/20151210_REU-St-Gilles_scuba_GOPR0510.JPG  
  inflating: /content/dataset/images/2024_dataset/20230704_REU-Cap-La-Houssaye_scuba_5_271.jpeg  
  inflating: /content/dataset/images/2024_dataset/20210830_MYT_scuba_11_00840.jpg  
  inflating: /content/dataset/images/2024_dataset/20210218_REU-Hermitage_scuba_5_00002.jpg  
  inflating

In [6]:
# all samples in original data
!ls /content/dataset/images/2024_dataset | wc -l

14492


In [7]:
!rm /content/dataset/images.zip

In [11]:
import pandas as pd
import os

labeled_images_df = pd.read_csv('/content/labeled_image_names.csv')
labeled_image_names = set(labeled_images_df[labeled_images_df.columns[0]].tolist())

image_directory = "/content/dataset/images/2024_dataset"
image_files_in_directory = set(os.listdir(image_directory))

count = len(labeled_image_names.intersection(image_files_in_directory))

print(f"{count} labelled files in {image_directory}")

1199 labelled files in /content/dataset/images/2024_dataset


In [ ]:
import os
import shutil
import pandas as pd

unlabelled_dir = "/content/dataset/unlabelled"
os.makedirs(unlabelled_dir, exist_ok=True)

# list of labeled images
labeled_images_df = pd.read_csv('/content/labeled_image_names.csv')
labeled_image_names = set(labeled_images_df[labeled_images_df.columns[0]].tolist())

image_directory = "/content/dataset/images/2024_dataset"
all_image_files = set(os.listdir(image_directory))

unlabeled_image_files = all_image_files - labeled_image_names

# Move the unlabeled image files to the new directory
print(f"Moving {len(unlabeled_image_files)} unlabelled images to {unlabelled_dir}...")
for file_name in unlabeled_image_files:
    source_path = os.path.join(image_directory, file_name)
    destination_path = os.path.join(unlabelled_dir, file_name)
    shutil.move(source_path, destination_path)

print("Move operation completed.")

Moving 13293 unlabelled images to /content/dataset/unlabelled...
Move operation completed.


In [13]:
# all labelled samples
!ls /content/dataset/images/2024_dataset | wc -l

1199


In [15]:
!mkdir -p /content/dataset/labelled

In [ ]:
import os
import shutil

labelled_dir = "/content/dataset/labelled"
os.makedirs(labelled_dir, exist_ok=True)

image_directory = "/content/dataset/images/2024_dataset"
all_items_in_directory = os.listdir(image_directory)

print(f"Moving images from {image_directory} to {labelled_dir}...")
for item_name in all_items_in_directory:
    source_path = os.path.join(image_directory, item_name)
    destination_path = os.path.join(labelled_dir, item_name)
    if os.path.isfile(source_path):
        shutil.move(source_path, destination_path)

print("Move operation completed.")

Moving images from /content/dataset/images/2024_dataset to /content/dataset/labelled...
Move operation completed.


In [18]:
!rm -rf /content/dataset/images/2024_dataset

In [19]:
# final: labelled
!ls /content/dataset/labelled | wc -l

1199


In [ ]:
# final: unlabelled
!ls /content/dataset/unlabelled/ | wc -l

13293
